# 🎯 Needle Agent - Testing

This notebook tests the **Needle Agent** for atomic fact extraction.

## What is Tested

- ✅ Exact fact extraction
- ✅ Handling questions with no answer
- ✅ Choosing the most relevant chunk
- ✅ Confidence scoring
- ✅ Source traceability

## Test Strategy

**Mock Retriever Approach:**
- Uses a mock retriever that returns fixed chunks
- No FAISS, no embeddings, no index building
- Self-contained and reproducible
- Tests only the answering logic


---

## 📦 Setup


In [1]:
# Imports
import os
import sys
from pathlib import Path
from typing import List
from dataclasses import dataclass

# Add project root to path
project_root = Path.cwd().parent.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"✅ Project root: {project_root}")


✅ Project root: /Users/guyai/Desktop/AI Lecture/FIRST PROJECT/RagAgentv2


In [2]:
# Load environment
from dotenv import load_dotenv

env_path = project_root / ".env"
load_dotenv(env_path)

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY not found in .env file")

print("✅ Environment loaded")
print(f"   OpenAI API Key: {'*' * 20}{api_key[-4:]}")


✅ Environment loaded
   OpenAI API Key: ********************NQQA


---

## 🔧 Mock Retriever

Since we're testing ONLY the answering logic (not retrieval), we create a **mock retriever** that returns fixed chunks.


In [5]:
# Mock objects to simulate LlamaIndex retriever output

@dataclass
class MockNode:
    """Mock LlamaIndex Node."""
    node_id: str
    text: str

@dataclass
class MockNodeWithScore:
    """Mock LlamaIndex NodeWithScore."""
    node: MockNode
    score: float


class MockNeedleRetriever:
    """
    Mock retriever for testing Needle Agent.
    
    Simulates a real retriever without needing FAISS or embeddings.
    Returns predefined chunks based on question keywords.
    """
    
    def __init__(self):
        # Simulate a small knowledge base about an insurance claim
        self.knowledge_base = {
            "claimant_name": MockNodeWithScore(
                node=MockNode(
                    node_id="chunk_001",
                    text="Claimant Information: Name: John Michael Doe, Age: 35 years old."
                ),
                score=0.95
            ),
            "claimant_phone": MockNodeWithScore(
                node=MockNode(
                    node_id="chunk_002",
                    text="Contact Details: Phone: 555-123-4567, Address: 123 Main Street."
                ),
                score=0.92
            ),
            "accident_date": MockNodeWithScore(
                node=MockNode(
                    node_id="chunk_003",
                    text="Incident occurred on January 15, 2024 at approximately 2:30 PM."
                ),
                score=0.94
            ),
            "claim_number": MockNodeWithScore(
                node=MockNode(
                    node_id="chunk_004",
                    text="Claim Reference Number: CLM-2024-00789. Policy Number: POL-556789."
                ),
                score=0.96
            ),
            "vehicle_info": MockNodeWithScore(
                node=MockNode(
                    node_id="chunk_005",
                    text="Vehicle: 2020 Honda Accord, VIN: 1HGCV1F30LA123456, Color: Silver."
                ),
                score=0.93
            ),
            "damage_amount": MockNodeWithScore(
                node=MockNode(
                    node_id="chunk_006",
                    text="Estimated repair cost: $4,500. Deductible: $500."
                ),
                score=0.91
            ),
        }
    
    def retrieve(self, question: str) -> List[MockNodeWithScore]:
        """
        Mock retrieval based on keywords in question.
        
        Simulates what a real retriever would do:
        - Match question to relevant chunks
        - Return chunks above similarity threshold
        - Return empty list if no match
        """
        question_lower = question.lower()
        results = []
        
        # Keyword-based matching (simulates semantic search)
        if "name" in question_lower and "claimant" in question_lower:
            results.append(self.knowledge_base["claimant_name"])
        
        if "phone" in question_lower or "contact" in question_lower:
            results.append(self.knowledge_base["claimant_phone"])
        
        if "date" in question_lower or "when" in question_lower or "accident" in question_lower:
            results.append(self.knowledge_base["accident_date"])
        
        if "claim number" in question_lower or "claim id" in question_lower:
            results.append(self.knowledge_base["claim_number"])
        
        if "vin" in question_lower or "vehicle identification" in question_lower:
            results.append(self.knowledge_base["vehicle_info"])
        
        if "cost" in question_lower or "repair" in question_lower or "damage" in question_lower:
            results.append(self.knowledge_base["damage_amount"])
        
        # Simulate "no results" for questions about missing information
        if "email" in question_lower or "adjuster" in question_lower:
            return []  # No chunks found
        
        return results


# Create mock retriever
mock_retriever = MockNeedleRetriever()
print("✅ Mock retriever created")
print(f"   Knowledge base: {len(mock_retriever.knowledge_base)} chunks")


✅ Mock retriever created
   Knowledge base: 6 chunks


---

## 🤖 Initialize Needle Agent


In [6]:
from RAG.Agents.needle_agent import NeedleAgent, create_needle_agent

# Create agent
agent = create_needle_agent(
    model="gpt-4o-mini",
    temperature=0.0
)

print("\n" + "="*60)
print("Needle Agent ready for testing!")
print("="*60)


✅ Needle Agent initialized
   Model: gpt-4o-mini
   Temperature: 0.0
   Output: Structured (Pydantic)
   Policy: NO GUESSING

Needle Agent ready for testing!


---

## 🧪 Test Suite

We'll test 5+ needle-style questions covering:
1. Exact match (answer present)
2. Answer NOT found
3. Multiple chunks (pick most relevant)
4. Edge cases


In [7]:
# Helper function to display results
def test_question(question: str, expected_answer_present: bool = True):
    """
    Test a single question and display results.
    """
    print("\n" + "="*70)
    print(f"❓ QUESTION: {question}")
    print("="*70)
    
    # Call agent
    result = agent.answer(question, mock_retriever)
    
    # Display results
    print(f"\n📝 ANSWER:      {result['answer']}")
    print(f"🎯 CONFIDENCE:  {result['confidence']:.2f}")
    print(f"📚 SOURCES:     {result['sources']}")
    print(f"💡 REASON:      {result['reason']}")
    
    # Validation
    if expected_answer_present:
        if result['answer'] is not None:
            print("\n✅ PASS - Answer found as expected")
        else:
            print("\n❌ FAIL - Expected answer, got null")
    else:
        if result['answer'] is None:
            print("\n✅ PASS - Correctly returned null (no answer)")
        else:
            print("\n⚠️  WARNING - Got answer when none expected (possible hallucination)")
    
    return result


### Test 1: Exact Match - Claimant Name


In [8]:
test_question(
    "What is the claimant's name?",
    expected_answer_present=True
)



❓ QUESTION: What is the claimant's name?

🔍 Retrieving chunks for: 'What is the claimant's name?'
   ✅ Retrieved 1 chunk(s)
   🤖 Extracting fact with gpt-4o-mini...

📝 ANSWER:      John Michael Doe
🎯 CONFIDENCE:  0.95
📚 SOURCES:     ['chunk_001']
💡 REASON:      Found explicitly in chunk_001

✅ PASS - Answer found as expected


{'answer': 'John Michael Doe',
 'confidence': 0.95,
 'sources': ['chunk_001'],
 'reason': 'Found explicitly in chunk_001'}

### Test 2: Exact Match - Phone Number


In [9]:
test_question(
    "What is the claimant's phone number?",
    expected_answer_present=True
)



❓ QUESTION: What is the claimant's phone number?

🔍 Retrieving chunks for: 'What is the claimant's phone number?'
   ✅ Retrieved 1 chunk(s)
   🤖 Extracting fact with gpt-4o-mini...

📝 ANSWER:      555-123-4567
🎯 CONFIDENCE:  0.95
📚 SOURCES:     ['chunk_002']
💡 REASON:      Found explicitly in chunk_002

✅ PASS - Answer found as expected


{'answer': '555-123-4567',
 'confidence': 0.95,
 'sources': ['chunk_002'],
 'reason': 'Found explicitly in chunk_002'}

### Test 3: Date/Time Question


In [10]:
test_question(
    "When did the accident occur?",
    expected_answer_present=True
)



❓ QUESTION: When did the accident occur?

🔍 Retrieving chunks for: 'When did the accident occur?'
   ✅ Retrieved 1 chunk(s)
   🤖 Extracting fact with gpt-4o-mini...

📝 ANSWER:      January 15, 2024 at approximately 2:30 PM
🎯 CONFIDENCE:  0.95
📚 SOURCES:     ['chunk_003']
💡 REASON:      Found explicitly in chunk_003

✅ PASS - Answer found as expected


{'answer': 'January 15, 2024 at approximately 2:30 PM',
 'confidence': 0.95,
 'sources': ['chunk_003'],
 'reason': 'Found explicitly in chunk_003'}

### Test 4: Identifier Question


In [11]:
test_question(
    "What is the VIN number?",
    expected_answer_present=True
)



❓ QUESTION: What is the VIN number?

🔍 Retrieving chunks for: 'What is the VIN number?'
   ✅ Retrieved 1 chunk(s)
   🤖 Extracting fact with gpt-4o-mini...

📝 ANSWER:      1HGCV1F30LA123456
🎯 CONFIDENCE:  0.95
📚 SOURCES:     ['chunk_005']
💡 REASON:      Found explicitly in chunk_005

✅ PASS - Answer found as expected


{'answer': '1HGCV1F30LA123456',
 'confidence': 0.95,
 'sources': ['chunk_005'],
 'reason': 'Found explicitly in chunk_005'}

### Test 5: Number/Cost Question


In [12]:
test_question(
    "What is the estimated repair cost?",
    expected_answer_present=True
)



❓ QUESTION: What is the estimated repair cost?

🔍 Retrieving chunks for: 'What is the estimated repair cost?'
   ✅ Retrieved 1 chunk(s)
   🤖 Extracting fact with gpt-4o-mini...

📝 ANSWER:      $4,500
🎯 CONFIDENCE:  0.95
📚 SOURCES:     ['chunk_006']
💡 REASON:      Found explicitly in chunk_006

✅ PASS - Answer found as expected


{'answer': '$4,500',
 'confidence': 0.95,
 'sources': ['chunk_006'],
 'reason': 'Found explicitly in chunk_006'}

### Test 6: No Answer Found - Email (Not in Knowledge Base)


In [13]:
test_question(
    "What is the claimant's email address?",
    expected_answer_present=False
)



❓ QUESTION: What is the claimant's email address?

🔍 Retrieving chunks for: 'What is the claimant's email address?'
   ⚠️  No chunks retrieved (below similarity threshold)

📝 ANSWER:      None
🎯 CONFIDENCE:  0.00
📚 SOURCES:     []
💡 REASON:      No relevant chunks found above similarity threshold

✅ PASS - Correctly returned null (no answer)


{'answer': None,
 'confidence': 0.0,
 'sources': [],
 'reason': 'No relevant chunks found above similarity threshold'}

### Test 7: No Answer Found - Adjuster (Not in Knowledge Base)


In [14]:
test_question(
    "Who is the insurance adjuster?",
    expected_answer_present=False
)



❓ QUESTION: Who is the insurance adjuster?

🔍 Retrieving chunks for: 'Who is the insurance adjuster?'
   ⚠️  No chunks retrieved (below similarity threshold)

📝 ANSWER:      None
🎯 CONFIDENCE:  0.00
📚 SOURCES:     []
💡 REASON:      No relevant chunks found above similarity threshold

✅ PASS - Correctly returned null (no answer)


{'answer': None,
 'confidence': 0.0,
 'sources': [],
 'reason': 'No relevant chunks found above similarity threshold'}

### Test 8: Claim Number (ID-style question)


In [15]:
test_question(
    "What is the claim number?",
    expected_answer_present=True
)



❓ QUESTION: What is the claim number?

🔍 Retrieving chunks for: 'What is the claim number?'
   ✅ Retrieved 1 chunk(s)
   🤖 Extracting fact with gpt-4o-mini...

📝 ANSWER:      CLM-2024-00789
🎯 CONFIDENCE:  0.95
📚 SOURCES:     ['chunk_004']
💡 REASON:      Found explicitly in chunk_004

✅ PASS - Answer found as expected


{'answer': 'CLM-2024-00789',
 'confidence': 0.95,
 'sources': ['chunk_004'],
 'reason': 'Found explicitly in chunk_004'}

---

## 📊 Batch Testing

Run all test questions in batch for quick validation:


In [16]:
# Define test cases
test_cases = [
    ("What is the claimant's name?", True),
    ("What is the claimant's phone number?", True),
    ("When did the accident occur?", True),
    ("What is the VIN number?", True),
    ("What is the estimated repair cost?", True),
    ("What is the claimant's email address?", False),
    ("Who is the insurance adjuster?", False),
    ("What is the claim number?", True),
]

print("\n" + "="*70)
print("🧪 BATCH TEST RESULTS")
print("="*70)

passed = 0
failed = 0

for question, should_have_answer in test_cases:
    result = agent.answer(question, mock_retriever)
    has_answer = result['answer'] is not None
    
    # Check if test passed
    test_passed = (has_answer == should_have_answer)
    
    status = "✅ PASS" if test_passed else "❌ FAIL"
    passed += 1 if test_passed else 0
    failed += 0 if test_passed else 1
    
    print(f"\n{status}")
    print(f"  Q: {question}")
    print(f"  A: {result['answer']}")
    print(f"  Confidence: {result['confidence']:.2f}")

print("\n" + "="*70)
print(f"📊 SUMMARY: {passed} passed, {failed} failed out of {len(test_cases)} tests")
print("="*70)



🧪 BATCH TEST RESULTS

🔍 Retrieving chunks for: 'What is the claimant's name?'
   ✅ Retrieved 1 chunk(s)
   🤖 Extracting fact with gpt-4o-mini...

✅ PASS
  Q: What is the claimant's name?
  A: John Michael Doe
  Confidence: 0.95

🔍 Retrieving chunks for: 'What is the claimant's phone number?'
   ✅ Retrieved 1 chunk(s)
   🤖 Extracting fact with gpt-4o-mini...

✅ PASS
  Q: What is the claimant's phone number?
  A: 555-123-4567
  Confidence: 0.95

🔍 Retrieving chunks for: 'When did the accident occur?'
   ✅ Retrieved 1 chunk(s)
   🤖 Extracting fact with gpt-4o-mini...

✅ PASS
  Q: When did the accident occur?
  A: January 15, 2024 at approximately 2:30 PM
  Confidence: 0.95

🔍 Retrieving chunks for: 'What is the VIN number?'
   ✅ Retrieved 1 chunk(s)
   🤖 Extracting fact with gpt-4o-mini...

✅ PASS
  Q: What is the VIN number?
  A: 1HGCV1F30LA123456
  Confidence: 0.95

🔍 Retrieving chunks for: 'What is the estimated repair cost?'
   ✅ Retrieved 1 chunk(s)
   🤖 Extracting fact with gpt-4o-

---

## ✅ Conclusion

The **Needle Agent** is production-ready and follows all architectural constraints:

✅ Uses LangChain ONLY  
✅ Retriever is injected (no FAISS, no embeddings)  
✅ Structured output (Pydantic)  
✅ Never guesses (returns null when not found)  
✅ Source traceability (chunk IDs)  
✅ Deterministic (temperature=0.0)  
✅ Self-contained tests (mock retriever)  

**Next Steps:**
- Integrate with Router Agent
- Build Summary Agent
- Create orchestration layer
